In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32


In [7]:


train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2

)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset ='training'

)
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

Found 1888 images belonging to 2 classes.
Found 472 images belonging to 2 classes.
Found 68 images belonging to 1 classes.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [10]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
history=model.fit(train_generator,validation_data=val_generator,epochs=1)

59/59 [==============================] - 130s 2s/step - loss: 0.2427 - accuracy: 0.9031 - val_loss: 0.2606 - val_accuracy: 0.9025


In [14]:
model.save("Model.h5","labe.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#Load the saved model
model=load_model('/content/Model.h5')

#Load and preprocess the test image
test_image_path='/content/drive/MyDrive/test/pred/pred1 (1).jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0) #Add batch dimension
img_array/=225. #Normalize the pixel values
#Make prediction
prediction=model.predict(img_array)
#print the prediction
if prediction<0.5:
  print("Prediction:Yes Tumor(Probability:",prediction[0][0],")")
else:
  print("Prediction:No Tumor(Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 99ms/step
Prediction:Yes Tumor(Probability: 0.017016172 )
